# A KÖNYVTÁRSZERKEZET VÁLTOZOTT 
.
└── DL_Project/
    ├── ACDC/
    │   ├── training/
    │   │   ├── patient001
    │   │   └── ...
    │   └── testing/
    │       ├── patient101
    │       └── ...
    └── ACDC_Resize/
        ├── training/
        │   ├── images
        │   ├── images_gt
        │   └── images_4d
        └── test/
            ├── images
            ├── images_gt
            └── images_4d

In [1]:
#!pip install nilearn
#!pip install dicom2nifti

import dicom2nifti
import nibabel as nib
import nilearn as nil
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
os.listdir('./')

['.git',
 '.gitignore',
 '.idea',
 '.ipynb_checkpoints',
 '.venv',
 'ACDC',
 'ACDC_Resize',
 'DATA_LOAD_TEST',
 'medreaders-master',
 'nifti-Copy1.ipynb',
 'nifti.ipynb',
 'patient_001_4d_30sec (2)',
 'patient_001_4d_30sec (2).zip',
 'project.ipynb',
 'README.md',
 'Untitled.ipynb',
 'Untitled1.ipynb',
 'venv']

## Load NIfTI file

We use `NiBabel` to read in the NIfTI file we just created:

In [3]:
#cardiac_vol_4d = nib.load('patient001_4d.nii.gz')
cardiac_vol1 = nib.load('ACDC/training/patient001/patient001_frame01.nii.gz')
#http://127.0.0.1:8890/edit/DL_Project/ACDC/training/patient001/patient001_frame01.nii.gz
cardiac_vol2 = nib.load('ACDC/training/patient001/patient001_frame01_gt.nii.gz')
cardiac_vol3 = nib.load('ACDC/training/patient001/patient001_frame12.nii.gz')
cardiac_vol4 = nib.load('ACDC/training/patient001/patient001_frame12_gt.nii.gz')

# What is the type of this object?
type(cardiac_vol1)

nibabel.nifti1.Nifti1Image

## Access data in the NIfTI object

NiBabel's handling of the NIfTI format data is not quite as elegant as what we saw in teh previous lesson. Rather than being able to access the data directly by referencing the name of the object (in this case, `cardiac_vol1`), we need to use the method `get_fdata()` to do this (the "f" in this method name stands for "floating point", as this is the type of data it returns). We will assign the result of this to a new variable so that it's easy to work with.

In [55]:
cardiac_vol_data = cardiac_vol1.get_fdata()
type(cardiac_vol_data)


numpy.ndarray

In [6]:

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os.path
    
# későbbi tanítás érdekében külön gyűjtöm a képeket és a hozzá tartozó gt-t
patient_vols_train =[]
patient_vols_train_gt =[]
patient_vols_test =[]
patient_vols_test_gt =[]
patient_vols_train_4d =[] # külön a 4d-s képek, kell ez nekünk ha nincs gt??
patient_vols_test_4d =[] # külön a 4d-s képek, kell ez nekünk ha nincs gt??


for i in range(1, 101):
    for frame_num in range(1, 17): #hogyha az adatot kéne beolvasni, akkot kell a végükre még egy .
        if os.path.isfile(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_frame{frame_num:02}.nii.gz'):
            patient_vols_train.append(nib.load(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_frame{frame_num:02}.nii.gz'))
        if os.path.isfile(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_frame{frame_num:02}_gt.nii.gz'):
            patient_vols_train_gt.append(nib.load(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_frame{frame_num:02}_gt.nii.gz'))

    if(os.path.isfile(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_4d.nii.gz')):     
        patient_vols_train_4d.append(nib.load(f'ACDC/training/patient{101-i:03}/patient{101-i:03}_4d.nii.gz'))
    
    
for i in range(101, 151):
    for frame_num in range(1, 17): #hogyha az adatot kéne beolvasni, akkot kell a végükre még egy .
        if os.path.isfile(f'ACDC/testing/patient{i}/patient{i}_frame{frame_num:02}.nii.gz'):
            patient_vols_test.append(nib.load(f'ACDC/testing/patient{i}/patient{i}_frame{frame_num:02}.nii.gz'))
        if os.path.isfile(f'ACDC/testing/patient{i}/patient{i}_frame{frame_num:02}_gt.nii.gz'):
            patient_vols_test_gt.append(nib.load(f'ACDC/testing/patient{i}/patient{i}_frame{frame_num:02}_gt.nii.gz'))

    if(os.path.isfile(f'ACDC/testing/patient{i}/patient{i}_4d.nii.gz')):     
        patient_vols_test_4d.append(nib.load(f'ACDC/testing/patient{i}/patient{i}_4d.nii.gz'))
    

In [ ]:
# image dimensions statistics

In [10]:
import statistics

shape_x = []
shape_y = []
shape_z = []
for i in range(1, len(patient_vols_train), 2):
    shape = patient_vols_train[i].shape
    shape_x.append(shape[0])
    shape_y.append(shape[1])
    shape_z.append(shape[2])
    #print(shape_x,shape_y,shape_z)
# most common values
print(statistics.mode(shape_x)) #256
print(statistics.mode(shape_y)) #216
print(statistics.mode(shape_z)) #10

    

216
256
10


In [11]:
shape_x = []
shape_y = []
shape_z = []
for i in range(1, len(patient_vols_test), 2):
    shape = patient_vols_test[i].shape
    shape_x.append(shape[0])
    shape_y.append(shape[1])
    shape_z.append(shape[2])
    #print(shape_x,shape_y,shape_z)
# most common values
print(statistics.mode(shape_x)) #256
print(statistics.mode(shape_y)) #216
print(statistics.mode(shape_z)) #10

216
256
10


In [12]:
import statistics

shape_x = []
shape_y = []
shape_z = []
shape_t = []
for i in range(1, len(patient_vols_train_4d), 2):
    shape = patient_vols_train_4d[i].shape
    shape_x.append(shape[0])
    shape_y.append(shape[1])
    shape_z.append(shape[2])
    shape_t.append(shape[3])
    #print(shape_x,shape_y,shape_z)
# most common values
print(statistics.mode(shape_x)) #256
print(statistics.mode(shape_y)) #216
print(statistics.mode(shape_z)) #10
print(statistics.mode(shape_t)) #30

216
256
10
30


# Resizing to 216* 256* 10
    Bilinear Interpolation(order==1):
Bilinear interpolation considers the four nearest neighbor pixels and computes the output pixel value as a weighted average of these neighbors.
It produces smoother results than nearest-neighbor interpolation and is often a good choice for general image resizing tasks.

(other order method--> increased computational cost)

In [13]:
import skimage.transform as skTrans

patient_train_resized =[]
patient_train_gt_resized =[]
patient_train_4d_resized =[]
patient_test_resized =[]
patient_test_gt_resized =[]
patient_test_4d_resized =[]



In [64]:
# max sizes
print(len(patient_vols_train))
print(len(patient_vols_train_gt))
print(len(patient_vols_train_4d))
print(len(patient_vols_test))
print(len(patient_vols_test_gt))
print(len(patient_vols_test_4d))



200
200
100
100
100
50


In [14]:
for i in range(0,50):
    patient_train_resized.append(skTrans.resize(patient_vols_train[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_gt_resized.append(skTrans.resize(patient_vols_train_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_4d_resized.append(skTrans.resize(patient_vols_train_4d[i].get_fdata(), (216,256,10,30), order=1, preserve_range=False,anti_aliasing=True))
    patient_test_resized.append(skTrans.resize(patient_vols_test[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_test_gt_resized.append(skTrans.resize(patient_vols_test_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_test_4d_resized.append(skTrans.resize(patient_vols_test_4d[i].get_fdata(), (216,256,10,30), order=1, preserve_range=False,anti_aliasing=True))
 

In [ ]:

img_arr = patient_vols_train[55].get_fdata()
img_arr = np.squeeze(img_arr)
print(img_arr)
np.save('test', img_arr)

In [164]:
type(patient_train_resized)

array([[7.9, 7.3, 7. , ..., 6. , 6. , 6. ],
       [7.9, 7.3, 7. , ..., 6. , 6. , 6. ],
       [7.9, 7.3, 7. , ..., 6. , 6. , 6. ],
       ...,
       [7.9, 7.3, 7. , ..., 6. , 6. , 6. ],
       [7.9, 7.3, 7. , ..., 6. , 6. , 6. ],
       [7.9, 7.3, 7. , ..., 6. , 6. , 6. ]])

In [171]:
patient_vols_train[0].get_fdata()[0]

array([[8., 7., 7., ..., 6., 6., 6.],
       [8., 7., 7., ..., 6., 6., 6.],
       [8., 7., 7., ..., 6., 6., 6.],
       ...,
       [8., 7., 7., ..., 6., 6., 6.],
       [8., 7., 7., ..., 6., 6., 6.],
       [8., 7., 7., ..., 6., 6., 6.]])

In [174]:
type(patient_train_resized)

list

In [16]:
for i in range(50,100):
    patient_train_resized.append(skTrans.resize(patient_vols_train[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_gt_resized.append(skTrans.resize(patient_vols_train_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_4d_resized.append(skTrans.resize(patient_vols_train_4d[i].get_fdata(), (216,256,10,30), order=1, preserve_range=False,anti_aliasing=True))
    patient_test_resized.append(skTrans.resize(patient_vols_test[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_test_gt_resized.append(skTrans.resize(patient_vols_test_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    

In [18]:
for i in range(100,150):
    patient_train_resized.append(skTrans.resize(patient_vols_train[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_gt_resized.append(skTrans.resize(patient_vols_train_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    


In [24]:
for i in range(150,200):
    patient_train_resized.append(skTrans.resize(patient_vols_train[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    patient_train_gt_resized.append(skTrans.resize(patient_vols_train_gt[i].get_fdata(), (216,256,10), order=1, preserve_range=False,anti_aliasing=True))
    


In [25]:
print(len(patient_train_resized))
print(len(patient_train_gt_resized))
print(len(patient_train_4d_resized))
print(len(patient_test_resized))
print(len(patient_test_gt_resized))
print(len(patient_test_4d_resized))

200
200
100
100
100
50


In [20]:
type(patient_train_resized)

list

In [21]:
type(patient_train_resized[0])



numpy.ndarray

In [22]:
patient_train_resized[3].shape

(216, 256, 10)

# save the numpy arrays 
# make folders: ACDC_Resize/training/images/
                                                

In [61]:
#parent_dir = 'test'## change only this

In [62]:
directory_path_1 = 'ACDC_Resize'
directory_path_2 = 'training'
directory_path_3 = 'test'
directory_path_4 = 'images'
directory_path_5 = 'images_gt'
directory_path_6 = 'images_4d'


In [63]:
#path = os.path.join(parent_dir, directory_path_1) 
os.mkdir(directory_path_1)

path_train = os.path.join(path, directory_path_2) 
os.mkdir(path_train)
path_test = os.path.join(path, directory_path_3) 
os.mkdir(path_test)

path_train_img = os.path.join(path_train, directory_path_4) 
os.mkdir(path_train_img)
path_test_img = os.path.join(path_test, directory_path_4) 
os.mkdir(path_test_img)

path_train_img = os.path.join(path_train, directory_path_5) 
os.mkdir(path_train_img)
path_test_img = os.path.join(path_test, directory_path_5) 
os.mkdir(path_test_img)

path_train_img = os.path.join(path_train, directory_path_6) 
os.mkdir(path_train_img)
path_test_img = os.path.join(path_test, directory_path_6) 
os.mkdir(path_test_img)

In [26]:
for i in range(150,200):
    np.save(f'ACDC_Resize/training/images/numpy_data_train_{i:03}.npy', patient_train_resized[i])
    np.save(f'ACDC_Resize/training/images_gt/numpy_data_train_gt_{i:03}.npy', patient_train_gt_resized[i])

# ellenőrzés - ok

In [27]:
np_train = np.load('ACDC_Resize/training/images/numpy_data_train_001.npy')

In [28]:
np_train

array([[[ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ],
        [ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ],
        [ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ],
        ...,
        [ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ],
        [ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ],
        [ 7.9     ,  7.3     ,  7.      , ...,  6.      ,  6.      ,
          6.      ]],

       [[ 8.078125,  7.421875,  8.59375 , ...,  7.59375 ,  7.21875 ,
          6.65625 ],
        [10.109375,  9.078125, 24.5625  , ..., 22.578125, 18.775   ,
         13.45    ],
        [10.05625 , 10.01875 , 26.9375  , ..., 21.5     , 18.2     ,
         14.6     ],
        ...,
        [11.84375 , 11.28125 ,  9.734375, ...,  8.03125 ,  8.34375 ,
          8.90625 ],
        [12.86875 , 11.14375 ,  9.5     , ...,  9.578125, 10.509375,
   

In [34]:
if (patient_train_resized[1].all()==np_train.all()):
    print(True)

True


In [36]:
patient_test_gt_resized[0]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [38]:
for i in range(0,100):
    np.save(f'ACDC_Resize/test/images/numpy_data_test_{i:03}.npy', patient_test_resized[i])
    np.save(f'ACDC_Resize/test/images_gt/numpy_data_test_gt_{i:03}.npy', patient_test_gt_resized[i])
    np.save(f'ACDC_Resize/training/images_4d/numpy_data_training_4d_{i:03}.npy', patient_train_4d_resized[i])
    

In [39]:
for i in range(0,50):
    np.save(f'ACDC_Resize/test/images_4d/numpy_data_test_4d_{i:02}.npy', patient_test_4d_resized[i])